Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Hernan"
COLLABORATORS = ""

---


# DS320 Spring 2023: Midterm Project

Due on Mon 04/03/23 at 8:00 AM

The focus of this assignment is data <i><b>cleaning, preparation, and basis calculations</i></b>: deal with numbers, datetime values, and text

You should review the pipeline of cleaning and preparing data I drew on the whiteboard and the "data cleaning" slide (try to be clear about 05 steps there) before working on this assignment.

You will clean tweets taken from https://github.com/thuydt02/HCQ for a text sentiment analysis problem.
You need to download the dataset from this link, upzip it and work on the `Full_Tweet_to_github.csv` file.
Do not upload the dataset to your Jupyter Luther since it is big. You need to download this notebook and work on it with Google Colab. I am trying to find out a way to upload the dataset to Jupyter Luther. If it is successful (likely), I will let you know. But be prepared to work in Google Colab.  

The dataset has roughly 164K tweets. These tweets were pulled from Twitter, satisfies:

1. created in the year 2020
2. has the key word "Hydroxychroloquine"

I use this dataset for my research. I want to analyze the reactions and opinions of social network users on using the medication "Hydroxychloroqine" to treat COVID-19 disease.

See more about the paper: https://arxiv.org/pdf/2201.00237.pdf

There are 10 tasks, each is worth 10 points.

Note: I will mannually grade your code, so no test cases will be provided, but I can give you the expectation of the outcomes for tasks as I can.


In [1]:
#setting up: You have to run this code cell first to compile helping functions.

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import re
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

nltk.download('vader_lexicon')

#functions for pre processing 

#---clean up html elements and entities: e.g. <html> </html> &nbsp;
def cleanHtml(sentence):
    #cleanr = re.compile('<.*?>')
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


#---function to clean the word of any punctuation or special characters
def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    cleaned = re.sub(r'[.|,|:|;|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

#---stop words removing 
stop_words = set(stopwords.words('english'))
stop_words.update(['  ', 'zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)


#--- sentence stemering
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
    

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/LC/manzhe01/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Task 01: Understand the data set
- Read the whole tweet dataset (Full_Tweet_to_github.csv file) into a dataframe. Name it `df_tweets`.
From now, you will work on `df_tweets`.
- Find out information about the follows. You must show your code:
    Shape

    Data types of all columns

    Numerical columns

    Text columns

    Categorical columns

    Date/time columns

    Statistics (min, max, mean, std, ...) of all the numerical columns
    
 Expectation: df_tweets.shape = (164168, 12)

In [2]:
df_tweets = pd.read_csv('Full_Tweet_to_github.csv')
print(df_tweets.shape)

(164168, 12)


# Task 02: Delete uneccessary columns
As you can see, the following columns will not help a machine learning algorithm to learn:
+ HYDROXYCHLOROQUINE: all the values in this columns are 1
+ query_string: the URL link to the tweet in Twitter.

Delete two above columns from `df_tweets`

expectaion: the shape of the returned dataframe = (164168, 10)

In [3]:
df_tweets.drop(['HYDROXYCHLOROQUINE', 'query_string'], axis=1, inplace=True)
print(df_tweets.shape)

(164168, 10)


# Task 03: Delete columns with missing values.

Delete all columns with more than 80% missing values.

In the previous assignment, you were asked to create a function, `delete_cols()`, call that function on `df_tweets` and `80`.

Expectation: df_tweets.shape = (164168, 9)


In [4]:
  
def delete_cols(df, threshold):
 
    missing_pct = df.isna().sum() / len(df)
    
    to_drop = missing_pct[missing_pct > threshold].index.tolist()
    
    df.drop(to_drop, axis=1, inplace=True)
    
    return df
df_tweets = delete_cols(df_tweets, 0.8)
print(df_tweets.shape)


(164168, 9)


# Task 04: Remove duplicates and irrelevant data

    a. Identify duplicates: 
        
        Two tweets are indentical if they have same 
        full_text, created_at, reply_count, favorites_count
        
        Two different tweets can have the same full_text as other people can re-tweet
        
        Note: trying to combine the above features may be expensive
    
    b. remove duplicates

Expectation: 1277 duplicates are deleted

In [5]:
duplicates = df_tweets.duplicated(subset=['full_text', 'created_at', 'reply_count', 'favorite_count'])
print(f"Number of duplicates: {duplicates.sum()}")
df_tweets = df_tweets[~duplicates].reset_index(drop=True)

Number of duplicates: 1277


# Task 05: Clean text

Do the following sub tasks for all values in each text columns:
+ lowcase
+ strip spaces (leading and trailing)

Call the functions you have done in the tasks 06 and 07 of the previous assignment.


In [6]:


for column in df_tweets.select_dtypes(include='object'):
    df_tweets[column] = df_tweets[column].str.lower()
    # Apply strip spaces transformation
    df_tweets[column] = df_tweets[column].str.strip()

print(df_tweets)



                                                full_text  created_at  \
0       from the january 2020 arthritis care &amp; res...  2020-01-03   
1       @thelancetrheum classic “bulls eye retinopathy...  2020-01-07   
2       dr. anil pareek -research has revealed that hy...  2020-01-12   
3       a randomised phase ii trial of hydroxychloroqu...  2020-01-13   
4       @geekyjules what are the side effects of the b...  2020-01-14   
...                                                   ...         ...   
162886  @ritatay29333372 @edhunter54 @specialistgi @je...  2020-12-30   
162887  @alienxperience @sarahksilverman @marcorubio i...  2020-12-30   
162888  @hcphtx @sonyiego @abc13houston @fox26houston ...  2020-12-30   
162889  @drtomfrieden the covid-19 response is medical...  2020-12-30   
162890  @drtomfrieden * reduce hospitalizations *\nicu...  2020-12-30   

           user_location  friends_count  followers_count  reply_count  \
0                    NaN              5           

# Task 06: Clean tweets

Do the following sub tasks for all values in the `full_text` column:
+ only keep alphabetical letters
+ remove HTML tags and entities
+ remove punctuations
+ remove stop words (words have no contribution for sentiment identification of sentences)
+ stemming words: replace a word with its original version since they have the same meaning and sentiment in a sentence. For example, `happiness` is derived from `happy` => we will replace `happiness` by `happy`

You are provided all the functions in the setting up cell. Call them for this task.

Note this task will take a while.

In [8]:
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

df_tweets['full_text'] = df_tweets['full_text'].apply(cleanHtml)

In [9]:
def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    cleaned = re.sub(r'[.|,|:|;|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

df_tweets['full_text'] = df_tweets['full_text'].apply(cleanPunc)

In [11]:
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
df_tweets['full_text'] = df_tweets['full_text'].apply(removeStopWords)

In [13]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
df_tweets['full_text'] = df_tweets['full_text'].apply(stemming)

# Task 07: Add a new column

Add a new column, called `state` and fill values in this column using the the following instructions.

We want to derive a state from `user_location`, but the data in `user_location` is very messy: some have cities' names (e.g., Albany), some have cities' names and state, ...

So I am including a look-up table file, called `state_full.csv`, here for you. In this file I have 2 columns: `shortState` and `city`. Whenever an user location has a short state name or a city in columns `shortState` or `city`, you will fill the `state` column with the short state.

For examples:

`user_location` = 'albany, usa' => `state` = 'NY'

`user_location` = 'boston, massachusetts' => `state` = 'MA'

`user_location` = 'ames, ia' => `state` = 'IA'

`user_location` = 'boston' => `state` = 'MA'

...
Note: this task will take long time (about 3-4 hours) to run

Expectation: 53712 short state names (NaN values are not counted) will be filled 



In [20]:
state_df = pd.read_csv('state_full.csv')
def getState(location):
    if isinstance(location, float):
        return ''
    for state in state_df['shortState']:
        if state.lower() in str(location).lower():
            return state
    for city in state_df['city']:
        if city.lower() in str(location).lower():
            return state_df.loc[state_df['city'].str.lower() == city.lower(), 'shortState'].values[0]
    return ''

df_tweets['state'] = df_tweets['user_location'].apply(getState)
state_counts = df_tweets['state'].value_counts()

print(state_counts)

      66603
CA    17616
IN    11299
OR     6952
IL     4971
TX     4376
NY     4338
MA     4299
FL     3745
NE     3622
PA     3430
LA     3401
CO     3285
MI     2432
AR     2375
WA     2254
AL     2147
NC     1949
OH     1556
VA     1439
MO     1277
GA     1060
AZ      987
ND      880
WI      591
UT      584
IA      557
NV      404
MD      379
KY      371
HI      367
ID      359
ME      351
NJ      340
OK      310
RI      296
DE      263
KS      238
TN      219
AK      164
NM      157
SC      141
MN      130
DC      113
MS       98
CT       75
NH       57
MT       12
SD        9
WV        8
WY        3
VT        2
Name: state, dtype: int64


# Task 08: count #tweets and sum up #favorite_count by date

Count number of tweets and sum up favorite_count by dates. Store the results in a dataframe, called `df_count_by_date`. Sort the dataframe in the descending order of tweet counts.

hint: use `groupby()` on `df_tweets` and then `agg()` with count for full_text and sum for favorite_count

expectation: df_count_by_date.shape = (315, 2)

In [22]:
df_tweets['created_at'] = pd.to_datetime(df_tweets['created_at'])

df_count_by_date = df_tweets.groupby(df_tweets['created_at'].dt.date).agg({'full_text': 'count', 'favorite_count': 'sum'})

df_count_by_date = df_count_by_date.sort_values(by='full_text', ascending=False)

print(df_count_by_date.shape)

(315, 2)


# Task 09: count #tweets and sum up #favorite_count by state

Count number of tweets and sum up favorite_count by states. Store the results in a dataframe, called `df_count_by_state`. Sort the dataframe in the descending order of tweet counts.

hint: use `groupby()` on `df_tweets` and then `agg()` with count for full_text and sum for favorite_count

expectation: df_count_by_state.shape = (51, 2)

In [23]:
df_count_by_state = df_tweets.groupby('state').agg({'full_text': 'count', 'favorite_count': 'sum'})

# Sort the DataFrame by tweet counts in descending order
df_count_by_state = df_count_by_state.sort_values(by='full_text', ascending=False)

# Print the resulting DataFrame
print(df_count_by_state.shape)

(52, 2)


# Task 10: Top 10 tweets

+ Find tweets in top 10 highest reply_count, ordering from the highest to the least.
+ Find tweets in top 10 highest retweet_count, ordering from the highest to the least.
+ Find tweets in top 10 highest favorite_count, ordering from the highest to the least.

In [44]:

top_10_favorite_count = df_tweets.sort_values(by='favorite_count', ascending=False).head(10)
print(top_10_favorite_count)


                                               full_text  created_at  \
484    hydroxychloroquine azithromycin taken together...  2020-03-21   
9323               hydroxychloroquine httpstcoymobdcfgxs  2020-05-19   
93441  trump kept telling us take hydroxychloroquine ...  2020-10-06   
13294  highly respected henry ford health system repo...  2020-07-07   
52696  trump receiving regenerons polyclonal antibody...  2020-10-02   
486    please dont take hydroxychloroquine plaquenil ...  2020-03-21   
90726  president receiving multiple medications notew...  2020-10-05   
70263                hydroxychloroquine stand back stand  2020-10-02   
16840  please watch highly respected dr harvey risch ...  2020-08-24   
14600  imagine child porn taken social media quickly ...  2020-07-29   

                     user_location  friends_count  followers_count  \
484                 washington, dc             50         85725414   
9323                  brooklyn, ny           3005          2276258 

In [45]:
top_10_retweet_count = df_tweets.sort_values(by='retweet_count', ascending=False).head(10)
print(top_10_retweet_count)


                                               full_text  created_at  \
484    hydroxychloroquine azithromycin taken together...  2020-03-21   
486    please dont take hydroxychloroquine plaquenil ...  2020-03-21   
93441  trump kept telling us take hydroxychloroquine ...  2020-10-06   
540    please spread hydroxychloroquine azithromycin ...  2020-03-21   
16840  please watch highly respected dr harvey risch ...  2020-08-24   
9323               hydroxychloroquine httpstcoymobdcfgxs  2020-05-19   
13294  highly respected henry ford health system repo...  2020-07-07   
12905  want ensure everyone understands gravity situa...  2020-07-03   
13030  hydroxychloroquine works worked whole time tha...  2020-07-04   
14600  imagine child porn taken social media quickly ...  2020-07-29   

                     user_location  friends_count  followers_count  \
484                 washington, dc             50         85725414   
486    republic of the philippines            335            27605 

In [43]:
top_10_reply_count = df_tweets.sort_values(by='reply_count', ascending=False).head(10)
print(top_10_reply_count)

                                               full_text  created_at  \
484    hydroxychloroquine azithromycin taken together...  2020-03-21   
15326  taking hydroxychloroquine treat coronavirus di...  2020-07-31   
12924  surprising new study found controversial antim...  2020-07-03   
13294  highly respected henry ford health system repo...  2020-07-07   
3531   listen 45 suggests untested hydroxychloroquine...  2020-04-06   
13399  obamas hydroxychloroquine 2008 httpstcoodnmltdcbu  2020-07-11   
15254  actually believe hydroxychloroquine probably e...  2020-07-30   
5495   trump says jesus could avoided crucifixion tak...  2020-04-12   
16840  please watch highly respected dr harvey risch ...  2020-08-24   
785    breaking man died wife icu ingested chloroquin...  2020-03-23   

                      user_location  friends_count  followers_count  \
484                  washington, dc             50         85725414   
15326                           NaN            745           3427